In [1]:
!pip install mutagen

/home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/.venv/bin/pip: line 2: /home/mox/Documents/coding_projects/personal projects/ipod_recover_rename/.venv/bin/python: No such file or directory


In [98]:
import pathlib
import os

import mutagen as mg
from mutagen.id3 import ID3
import base64
from mutagen.oggvorbis import OggVorbis

import itertools
import shutil


In [3]:
cwd = pathlib.Path.cwd()

if cwd.name == 'ipod_recover_rename':
    print("Path is project root")
else:
    print('Current Working Directory is:', cwd)
    print("Recomend navigating to the project root in terminal and opening your chosen idea via the command 'code .'")
    raise NameError('Please correct current working directory to the project root')


Path is project root


In [4]:
old_path = cwd / "Music"
after_path = cwd / "After"

In [5]:
if not os.path.exists(after_path): 
    # if the After directory is not present  
    # then create it. 
    os.makedirs(after_path) 

In [6]:
f_list = os.listdir(old_path)
# copies music to after folder for processing of a copy of the music, ensuring no loss
for file in f_list:
    # prevents copying of the hidden files associated with ituns and ipods
    if file.startswith('._'):
        pass
    else:
        shutil.copytree(old_path / file, after_path / file)


In [7]:
# https://stackoverflow.com/questions/17547273/flatten-complex-directory-structure-in-python
#Move all files from music to (created) intermediary
def move(destination):
    all_files = []
    first_loop_pass = True
    for root, _dirs, files in os.walk(destination):
        if first_loop_pass:
            first_loop_pass = False
            continue
        for filename in files:
            all_files.append(os.path.join(root, filename))
    for filename in all_files:
        shutil.move(filename, destination)

In [8]:
move(after_path)

In [9]:
walk = list(os.walk(after_path))
for path, _, _ in walk[::]:
    if len(os.listdir(path)) == 0:
        # remove empty directories
        os.removedirs(path)


In [86]:
test_mp3 = after_path / 'CUYL.mp3'
test_m4a = after_path / 'XWHW.m4a'
test_m4p = after_path / 'DMZZ.m4p'
test_flac = after_path / 'ASCS.flac'
test_wav = after_path / 'GSDE.wav'
test_wma = after_path / 'AFES.wma'
test_move = after_path / 'tree_test.mp3'

test_files = [test_mp3, test_m4a, test_m4p, test_flac, test_wav, test_wma, test_move]

In [ ]:
# List all file types for appropriate parsing
compatible_types = {
    'id3': ('.mp3', '.wav'),
    'flac' : ('.flac'),
    'other': ('.m4a', '.m4p', '.wma',)
    #TODO: add support for other using mugagen tag readers schemes
}


#Potential restructure. Set sucessful = false. iterate across via try_id3(), try_APEv2() functions until a successful return.
def get_rename_dict(file):
    file_ext = file.suffix
    file_path = after_path / file
    f = mg.File(file_path)
    #handle ID3 compatible filetypes
    if file_ext in compatible_types['id3']:
        # Not stritly id3, but picks up on id3 and less strict id3 formats
        id3_file = mg.File(file_path)
        return {'artist':f['TPE1'].text[0],
                'album':f['TALB'].text[0],
                'title':f['TIT2'].text[0]}
    # handle flac
    if file_ext in compatible_types['flac']:
        flac_file = mg.flac.FLAC(file_path)
        # these tags are capitalization agnostic
        return {'artist':f['Artist'][0],
                'album':f['Album'][0],
                'title':f['Title'][0]}

    #TODO add suport for others in compatible types dict
    

In [130]:
def move_and_rename(file, name_dict):
    name_dict = get_rename_dict(after_path / file) 
    # build folder and file paths
    artist_path = after_path / name_dict['artist']
    album_path = artist_path /  name_dict['album']
    title = name_dict['title']
    # check if album-artist folder paths exist, if not make them.
    # if artist folder DNE, make it
    if not os.path.isdir(artist_path):
        os.mkdir(artist_path)
    # if album folder DNE, make it
    if not os.path.isdir(album_path):
        os.mkdir(album_path)
    # move the song to the album-artist folder and rename it
    shutil.move(after_path / file, album_path / title)

In [131]:
for file in test_files:
    try:
        name_dict = get_rename_dict(file) 
        move_and_rename(file, name_dict)
        print(file.name, 'successfull')
    except:
        print(file.name, "did not work", file.suffix, "is likely unsupported")

CUYL.mp3 successfull
XWHW.m4a did not work .m4a is likely unsupported
DMZZ.m4p did not work .m4p is likely unsupported
ASCS.flac successfull
GSDE.wav successfull
AFES.wma did not work .wma is likely unsupported
tree_test.mp3 successfull


TODO: 

1. rename files to artist_album_title.ext
2. Add files to artist-album-song structure
    1. walk file list and make [artist, album, title.ext]
    2. check if artist-album folders exist, make them if DNE 
    3. rename to title.ext and move to appropriate folder tree

In [125]:
name_dict = get_rename_dict(test_flac) 
move_and_rename(file, name_dict)
print(file.name, 'successfull')

{'artist': 'The Beatles', 'album': 'Rubber Soul', 'title': 'Drive My Car'}


MutagenError: [Errno 2] No such file or directory: '/home/mox/Documents/coding_projects/personal_projects/ipod_recover_rename/After/tree_test.mp3'